In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c dogs-vs-cats

dogs-vs-cats.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
from zipfile import ZipFile
import os

# Define the zip file path
zip_file_path = "/content/dogs-vs-cats.zip"

# Extract the dataset
with ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall("/content/dogs-vs-cats/")
    print("Extraction done!")


Extraction done!


In [ ]:
import cv2
import numpy as np
from sklearn.preprocessing import StandardScaler

In [ ]:
# Paths to the zip files
train_zip_path = "/content/dogs-vs-cats/train.zip"
test_zip_path = "/content/dogs-vs-cats/test1.zip"

# Define the extraction directories
train_dir = "/content/dogs-vs-cats/train/train"
test_dir = "/content/dogs-vs-cats/test/test1"

# Extract the training data
with ZipFile(train_zip_path, 'r') as zip_ref:
    zip_ref.extractall(train_dir)
    print("Training data extraction done!")

# Extract the testing data
with ZipFile(test_zip_path, 'r') as zip_ref:
    zip_ref.extractall(test_dir)
    print("Testing data extraction done!")


import os
path, dirs, files = next(os.walk("/content/kagglecatsanddogs_3367a/PetImages/Cat"))
print(len(files))


Training data extraction done!
Testing data extraction done!


In [ ]:
img_size = 64


def load_data(data_dir,num_samples=None):

    data = []
    labels = []
    for img_name in os.listdir(data_dir):
        img_path = os.path.join(data_dir, img_name)
        if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')):  # Ensure it's an image file
            continue
        label = 0 if 'cat' in img_name else 1  # Label: 0 for 'cat', 1 for 'dog'
        try:
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            if img is None:
                print(f"Failed to load image: {img_path}")
                continue
            img = cv2.resize(img, (img_size, img_size))
            data.append(img)
            labels.append(label)
        except Exception as e:
            print(f"Error loading image {img_name}: {e}")
    return np.array(data), np.array(labels)

num_samples = 600


# X_train, y_train = load_data(train_dir)
X_train, y_train = load_data(train_dir, num_samples=num_samples)
X_test, y_test = load_data(test_dir, num_samples=num_samples)

# Normalize the image data
X_train = X_train / 255.0

X_test, _ = load_data(test_dir)

# Normalize the image data
X_test = X_test / 255.0

In [ ]:
# Flatten the image data for SVM
X_train = X_train.reshape(len(X_train), -1)
X_test = X_test.reshape(len(X_test), -1)


In [ ]:
print(f"Number of images loaded for training: {len(X_train)}")
print(f"Number of images loaded for testing: {len(X_test)}")

Number of images loaded for training: 25000
Number of images loaded for testing: 12500


In [ ]:
from sklearn.preprocessing import StandardScaler

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training data and transform it
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data with the same scaler
X_test_scaled = scaler.transform(X_test)


In [ ]:
from sklearn.svm import SVC

# Initialize the SVM model with a linear kernel
svm = SVC(kernel='linear', random_state=42)

# Train the SVM model
svm.fit(X_train_scaled, y_train)


In [ ]:
import numpy as np
import os
import cv2
import random
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Define the image size
img_size = 64
num_samples = 3000 # Number of images to use

def get_image_files(data_dir, num_samples=None):
    """Retrieve image file paths from a directory."""
    if not os.path.isdir(data_dir):
        print(f"Directory {data_dir} does not exist.")
        return []

    img_files = [os.path.join(data_dir, img_name) for img_name in os.listdir(data_dir) if img_name.lower().endswith(('.png', '.jpg', '.jpeg'))]
    print(f"Found {len(img_files)} image files in {data_dir}.")
    if num_samples:
        img_files = random.sample(img_files, min(num_samples, len(img_files)))
    return img_files

def load_images(image_paths, img_size=64):
    """Load and preprocess images from file paths."""
    data = []
    labels = []
    for img_path in image_paths:
        label = 0 if 'cat' in os.path.basename(img_path) else 1  # Label: 0 for 'cat', 1 for 'dog'
        try:
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            if img is None:
                print(f"Failed to load image: {img_path}")
                continue
            img = cv2.resize(img, (img_size, img_size))
            data.append(img)
            labels.append(label)
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")

    print(f"Loaded {len(data)} images")
    return np.array(data), np.array(labels)

# Directories for your existing train and test data
train_dir = "/content/dogs-vs-cats/train/train"
test_dir = "/content/dogs-vs-cats/test/test1"

# Print the contents of directories
print("Checking contents of the train directory...")
print("Files in train directory:", os.listdir(train_dir)[:10])  # Print first 10 files

print("Checking contents of the test directory...")
print("Files in test directory:", os.listdir(test_dir)[:10])  # Print first 10 files

# Get paths of a subset of 600 images for training and testing
train_image_paths = get_image_files(train_dir, num_samples=num_samples)
test_image_paths = get_image_files(test_dir, num_samples=num_samples)

# Print out a few image paths to verify
print("Sample train image paths:", train_image_paths[:5])
print("Sample test image paths:", test_image_paths[:5])

# Load the selected images
X_train, y_train = load_images(train_image_paths, img_size=img_size)
X_test, y_test = load_images(test_image_paths, img_size=img_size)

# Print the number of images loaded
print(f"Number of images loaded for training: {len(X_train)}")
print(f"Number of images loaded for testing: {len(X_test)}")

# Check shapes of loaded images
print(f"Shape of X_train before flattening: {X_train.shape}")
print(f"Shape of X_test before flattening: {X_test.shape}")

if len(X_train) == 0 or len(X_test) == 0:
    print("Error: No images were loaded. Check the directory paths and image formats.")
else:
    # Normalize the image data
    X_train = X_train / 255.0
    X_test = X_test / 255.0

    # Flatten the image data for SVM
    X_train = X_train.reshape(len(X_train), -1)
    X_test = X_test.reshape(len(X_test), -1)

    # Check shapes of flattened images
    print(f"Shape of X_train after flattening: {X_train.shape}")
    print(f"Shape of X_test after flattening: {X_test.shape}")

    # Feature scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Initialize and train the SVM model
    svm = SVC(kernel='linear', random_state=42)
    svm.fit(X_train_scaled, y_train)

    # Make predictions on the test data
    y_pred = svm.predict(X_test_scaled)

    # Calculate and print the accuracy of the model
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy * 100:.2f}%")

    # Print the classification report
    print("Classification Report:\n", classification_report(y_test, y_pred, target_names=['cat', 'dog']))


Checking contents of the train directory...
Files in train directory: ['cat.5336.jpg', 'dog.6502.jpg', 'cat.7577.jpg', 'dog.7889.jpg', 'cat.1133.jpg', 'dog.1949.jpg', 'dog.5906.jpg', 'cat.1977.jpg', 'cat.2972.jpg', 'cat.6872.jpg']
Checking contents of the test directory...
Files in test directory: ['11726.jpg', '8610.jpg', '1591.jpg', '5412.jpg', '7290.jpg', '2023.jpg', '4866.jpg', '11290.jpg', '11426.jpg', '10373.jpg']
Found 25000 image files in /content/dogs-vs-cats/train/train.
Found 12500 image files in /content/dogs-vs-cats/test/test1.
Sample train image paths: ['/content/dogs-vs-cats/train/train/dog.3312.jpg', '/content/dogs-vs-cats/train/train/dog.4398.jpg', '/content/dogs-vs-cats/train/train/dog.3203.jpg', '/content/dogs-vs-cats/train/train/dog.10489.jpg', '/content/dogs-vs-cats/train/train/dog.5817.jpg']
Sample test image paths: ['/content/dogs-vs-cats/test/test1/5173.jpg', '/content/dogs-vs-cats/test/test1/4506.jpg', '/content/dogs-vs-cats/test/test1/6378.jpg', '/content/dogs

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
